This Python script performs instance segmentation using a pre-trained YOLO (You Only Look Once) model with Ultralytics.

**Ultralytics** is the creator of **YOLOv5** and **YOLOv8**, widely used for object detection, segmentation, and other computer vision tasks.

It loads the trained segmentation model, processes the input image or video, runs inference, and displays the segmented objects with masks and bounding boxes.


## Loading Dataset in google colab from gdrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import os

# Path to your zip file in Google Drive
zip_path = '/content/drive/MyDrive/coco2017_subset.zip'

# Directory where you want to extract the contents
extract_path = '/content/extracted_files'

# Create the directory if it doesn't exist
os.makedirs(extract_path, exist_ok=True)

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Files extracted to {extract_path}")

Files extracted to /content/extracted_files


## Importing required libraries

In [ ]:
import os
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import pandas as pd
import cv2
import seaborn as sns
import random
from skimage import io
import albumentations as A
import json

## Annotation generation for Train data

In [ ]:
# Path to COCO annotation file and output folder for YOLO labels
coco_ann_file = r"/content/extracted_files/coco2017_subset/annotation_subset/instances_train2017_subset.json"
output_dir = "/content/extracted_files/coco2017_subset/train2017"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Load COCO annotations
with open(coco_ann_file, 'r') as f:
    coco_data = json.load(f)

In [ ]:
# Create mapping: COCO category id -> new id (0-based contiguous indices)
# For example, if valid categories are 1, 2, 3, 5, 6, ... then you might have:
coco_categories = sorted([cat['id'] for cat in coco_data['categories']])
id_mapping = {orig_id: new_id for new_id, orig_id in enumerate(coco_categories)}

In [ ]:
# Build a mapping from image_id to image info
img_info = {img['id']: img for img in coco_data['images']}

In [ ]:
# Process annotations per image
annotations_by_image = {}
for ann in coco_data['annotations']:
    img_id = ann['image_id']
    img = img_info[img_id]
    img_width, img_height = img['width'], img['height']

    x, y, w, h = ann['bbox']
    # Convert COCO bbox [x, y, w, h] to YOLO format [x_center, y_center, width, height] normalized
    x_center = (x + w / 2) / img_width
    y_center = (y + h / 2) / img_height
    norm_w = w / img_width
    norm_h = h / img_height

    # Remap the category id
    orig_cat = ann['category_id']
    if orig_cat not in id_mapping:
        continue  # skip if the category is not in the mapping
    new_cat = id_mapping[orig_cat]

    if img_id not in annotations_by_image:
        annotations_by_image[img_id] = []
    annotations_by_image[img_id].append(f"{new_cat} {x_center:.6f} {y_center:.6f} {norm_w:.6f} {norm_h:.6f}")

In [ ]:
# Write YOLO label files for each image
for img_id, ann_list in annotations_by_image.items():
    file_name = img_info[img_id]['file_name']
    base_name = os.path.splitext(file_name)[0]
    # Save the label file in the same directory as your images, or wherever you prefer
    out_file = os.path.join(output_dir, base_name + '.txt')
    with open(out_file, 'w') as f:
        for line in ann_list:
            f.write(line + '\n')

## Annotation generation for Validation data

In [ ]:
coco_ann_file = '/content/extracted_files/coco2017_subset/annotation_subset/instances_val2017_subset.json'
output_dir = '/content/extracted_files/coco2017_subset/val2017'
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Load COCO annotations
with open(coco_ann_file, 'r') as f:
    coco_data = json.load(f)

In [ ]:
# Create mapping: COCO category id -> new id (0-based contiguous indices)
# For example, if valid categories are 1, 2, 3, 5, 6, ... then you might have:
coco_categories = sorted([cat['id'] for cat in coco_data['categories']])
id_mapping = {orig_id: new_id for new_id, orig_id in enumerate(coco_categories)}

In [ ]:
# Build a mapping from image_id to image info
img_info = {img['id']: img for img in coco_data['images']}

In [ ]:
# Process annotations per image
annotations_by_image = {}
for ann in coco_data['annotations']:
    img_id = ann['image_id']
    img = img_info[img_id]
    img_width, img_height = img['width'], img['height']

    x, y, w, h = ann['bbox']
    # Convert COCO bbox [x, y, w, h] to YOLO format [x_center, y_center, width, height] normalized
    x_center = (x + w / 2) / img_width
    y_center = (y + h / 2) / img_height
    norm_w = w / img_width
    norm_h = h / img_height

    # Remap the category id
    orig_cat = ann['category_id']
    if orig_cat not in id_mapping:
        continue  # skip if the category is not in the mapping
    new_cat = id_mapping[orig_cat]

    if img_id not in annotations_by_image:
        annotations_by_image[img_id] = []
    annotations_by_image[img_id].append(f"{new_cat} {x_center:.6f} {y_center:.6f} {norm_w:.6f} {norm_h:.6f}")

In [ ]:
# Write YOLO label files for each image
for img_id, ann_list in annotations_by_image.items():
    file_name = img_info[img_id]['file_name']
    base_name = os.path.splitext(file_name)[0]
    # Save the label file in the same directory as your images, or wherever you prefer
    out_file = os.path.join(output_dir, base_name + '.txt')
    with open(out_file, 'w') as f:
        for line in ann_list:
            f.write(line + '\n')

### Force UTF-8 Encoding

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

### Updation of YOLO Labels for Train dataset

In [ ]:
from pycocotools import mask as mask_utils

# Path to COCO annotation file and output folder for YOLO labels
coco_ann_file = r"/content/extracted_files/coco2017_subset/annotation_subset/instances_train2017_subset.json"
output_dir = "/content/extracted_files/coco2017_subset/train2017"
os.makedirs(output_dir, exist_ok=True)

# Load COCO annotations
with open(coco_ann_file, 'r') as f:
    coco_data = json.load(f)

# Create mapping: COCO category id -> new id (0-based contiguous indices)
coco_categories = sorted([cat['id'] for cat in coco_data['categories']])
id_mapping = {orig_id: new_id for new_id, orig_id in enumerate(coco_categories)}

# Build a mapping from image_id to image info
img_info = {img['id']: img for img in coco_data['images']}

# Process annotations per image
annotations_by_image = {}

# Process each annotation
for ann in coco_data['annotations']:
    try:
        img_id = ann['image_id']
        if img_id not in img_info:
            print(f"Warning: Image ID {img_id} not found in image info")
            continue

        img = img_info[img_id]
        img_width, img_height = float(img['width']), float(img['height'])

        # Get the category ID and check if it's in our mapping
        orig_cat = ann['category_id']
        if orig_cat not in id_mapping:
            continue  # skip if the category is not in the mapping
        new_cat = id_mapping[orig_cat]

        # Initialize the annotation list for this image if it doesn't exist
        if img_id not in annotations_by_image:
            annotations_by_image[img_id] = []

        # First, add the bounding box information (required for YOLO segmentation)
        x, y, w, h = map(float, ann['bbox'])  # Convert to float
        x_center = (x + w / 2) / img_width
        y_center = (y + h / 2) / img_height
        norm_w = w / img_width
        norm_h = h / img_height

        # Handle segmentation data
        polygons = []

        if 'segmentation' in ann:
            seg_data = ann['segmentation']

            # Check segmentation format
            if isinstance(seg_data, list):
                # It could be a list of lists (polygons) or a list of dicts (RLE)
                for seg in seg_data:
                    if isinstance(seg, list) and len(seg) >= 6:  # Polygon format (at least 3 points)
                        # Check if all elements are numbers
                        if all(isinstance(p, (int, float)) or (isinstance(p, str) and p.replace('.', '', 1).isdigit()) for p in seg):
                            polygon = []
                            for i in range(0, len(seg), 2):
                                if i+1 < len(seg):
                                    x_norm = float(seg[i]) / img_width
                                    y_norm = float(seg[i+1]) / img_height
                                    polygon.extend([x_norm, y_norm])
                            polygons.append(polygon)
                        else:
                            print(f"Warning: Non-numeric values in polygon for annotation {ann['id']}")
                    elif isinstance(seg, dict) and 'counts' in seg and 'size' in seg:
                        # RLE format
                        try:
                            # Convert RLE to binary mask
                            rle = mask_utils.frPyObjects(seg, img_height, img_width)
                            binary_mask = mask_utils.decode(rle)

                            # Find contours in the binary mask
                            contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                            # Convert contours to normalized polygon points
                            for contour in contours:
                                if len(contour) >= 3:  # At least 3 points needed
                                    polygon = []
                                    for point in contour:
                                        x_norm = float(point[0][0]) / img_width
                                        y_norm = float(point[0][1]) / img_height
                                        polygon.extend([x_norm, y_norm])
                                    polygons.append(polygon)
                        except Exception as e:
                            print(f"Error processing RLE data: {e}")
            elif isinstance(seg_data, dict) and 'counts' in seg_data and 'size' in seg_data:
                # Single RLE format
                try:
                    # Convert RLE to binary mask
                    rle = mask_utils.frPyObjects(seg_data, img_height, img_width)
                    binary_mask = mask_utils.decode(rle)

                    # Find contours in the binary mask
                    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                    # Convert contours to normalized polygon points
                    for contour in contours:
                        if len(contour) >= 3:  # At least 3 points needed
                            polygon = []
                            for point in contour:
                                x_norm = float(point[0][0]) / img_width
                                y_norm = float(point[0][1]) / img_height
                                polygon.extend([x_norm, y_norm])
                            polygons.append(polygon)
                except Exception as e:
                    print(f"Error processing RLE data: {e}")

        # Format the YOLO segmentation line for each polygon
        if polygons:
            for polygon in polygons:
                if len(polygon) >= 6:  # Minimum 3 points for a polygon (x1,y1,x2,y2,x3,y3)
                    line = f"{new_cat} {x_center:.6f} {y_center:.6f} {norm_w:.6f} {norm_h:.6f}"
                    for p in polygon:
                        line += f" {p:.6f}"
                    annotations_by_image[img_id].append(line)
        else:
            print(f"Warning: No valid polygon data for annotation {ann['id']} in image {img_id}")
    except Exception as e:
        print(f"Error processing annotation {ann.get('id', 'unknown')}: {e}")

# Write YOLO label files for each image
count = 0
for img_id, ann_list in annotations_by_image.items():
    if img_id in img_info and ann_list:  # Make sure we have info and annotations
        file_name = img_info[img_id]['file_name']
        base_name = os.path.splitext(file_name)[0]
        out_file = os.path.join(output_dir, base_name + '.txt')

        with open(out_file, 'w') as f:
            for line in ann_list:
                f.write(line + '\n')
        count += 1

print(f"Processed {count} images with segmentation annotations")

Processed 29315 images with segmentation annotations


### Updation of YOLO Labels for Validation dataset

In [ ]:
from pycocotools import mask as mask_utils


# Path to COCO annotation file and output folder for YOLO labels
coco_ann_file = r"/content/extracted_files/coco2017_subset/annotation_subset/instances_val2017_subset.json"
output_dir = "/content/extracted_files/coco2017_subset/val2017"
os.makedirs(output_dir, exist_ok=True)

# Load COCO annotations
with open(coco_ann_file, 'r') as f:
    coco_data = json.load(f)

# Create mapping: COCO category id -> new id (0-based contiguous indices)
coco_categories = sorted([cat['id'] for cat in coco_data['categories']])
id_mapping = {orig_id: new_id for new_id, orig_id in enumerate(coco_categories)}

# Build a mapping from image_id to image info
img_info = {img['id']: img for img in coco_data['images']}

# Process annotations per image
annotations_by_image = {}

# Process each annotation
for ann in coco_data['annotations']:
    try:
        img_id = ann['image_id']
        if img_id not in img_info:
            print(f"Warning: Image ID {img_id} not found in image info")
            continue

        img = img_info[img_id]
        img_width, img_height = float(img['width']), float(img['height'])

        # Get the category ID and check if it's in our mapping
        orig_cat = ann['category_id']
        if orig_cat not in id_mapping:
            continue  # skip if the category is not in the mapping
        new_cat = id_mapping[orig_cat]

        # Initialize the annotation list for this image if it doesn't exist
        if img_id not in annotations_by_image:
            annotations_by_image[img_id] = []

        # First, add the bounding box information (required for YOLO segmentation)
        x, y, w, h = map(float, ann['bbox'])  # Convert to float
        x_center = (x + w / 2) / img_width
        y_center = (y + h / 2) / img_height
        norm_w = w / img_width
        norm_h = h / img_height

        # Handle segmentation data
        polygons = []

        if 'segmentation' in ann:
            seg_data = ann['segmentation']

            # Check segmentation format
            if isinstance(seg_data, list):
                # It could be a list of lists (polygons) or a list of dicts (RLE)
                for seg in seg_data:
                    if isinstance(seg, list) and len(seg) >= 6:  # Polygon format (at least 3 points)
                        # Check if all elements are numbers
                        if all(isinstance(p, (int, float)) or (isinstance(p, str) and p.replace('.', '', 1).isdigit()) for p in seg):
                            polygon = []
                            for i in range(0, len(seg), 2):
                                if i+1 < len(seg):
                                    x_norm = float(seg[i]) / img_width
                                    y_norm = float(seg[i+1]) / img_height
                                    polygon.extend([x_norm, y_norm])
                            polygons.append(polygon)
                        else:
                            print(f"Warning: Non-numeric values in polygon for annotation {ann['id']}")
                    elif isinstance(seg, dict) and 'counts' in seg and 'size' in seg:
                        # RLE format
                        try:
                            # Convert RLE to binary mask
                            rle = mask_utils.frPyObjects(seg, img_height, img_width)
                            binary_mask = mask_utils.decode(rle)

                            # Find contours in the binary mask
                            contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                            # Convert contours to normalized polygon points
                            for contour in contours:
                                if len(contour) >= 3:  # At least 3 points needed
                                    polygon = []
                                    for point in contour:
                                        x_norm = float(point[0][0]) / img_width
                                        y_norm = float(point[0][1]) / img_height
                                        polygon.extend([x_norm, y_norm])
                                    polygons.append(polygon)
                        except Exception as e:
                            print(f"Error processing RLE data: {e}")
            elif isinstance(seg_data, dict) and 'counts' in seg_data and 'size' in seg_data:
                # Single RLE format
                try:
                    # Convert RLE to binary mask
                    rle = mask_utils.frPyObjects(seg_data, img_height, img_width)
                    binary_mask = mask_utils.decode(rle)

                    # Find contours in the binary mask
                    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                    # Convert contours to normalized polygon points
                    for contour in contours:
                        if len(contour) >= 3:  # At least 3 points needed
                            polygon = []
                            for point in contour:
                                x_norm = float(point[0][0]) / img_width
                                y_norm = float(point[0][1]) / img_height
                                polygon.extend([x_norm, y_norm])
                            polygons.append(polygon)
                except Exception as e:
                    print(f"Error processing RLE data: {e}")

        # Format the YOLO segmentation line for each polygon
        if polygons:
            for polygon in polygons:
                if len(polygon) >= 6:  # Minimum 3 points for a polygon (x1,y1,x2,y2,x3,y3)
                    line = f"{new_cat} {x_center:.6f} {y_center:.6f} {norm_w:.6f} {norm_h:.6f}"
                    for p in polygon:
                        line += f" {p:.6f}"
                    annotations_by_image[img_id].append(line)
        else:
            print(f"Warning: No valid polygon data for annotation {ann['id']} in image {img_id}")
    except Exception as e:
        print(f"Error processing annotation {ann.get('id', 'unknown')}: {e}")

# Write YOLO label files for each image
count = 0
for img_id, ann_list in annotations_by_image.items():
    if img_id in img_info and ann_list:  # Make sure we have info and annotations
        file_name = img_info[img_id]['file_name']
        base_name = os.path.splitext(file_name)[0]
        out_file = os.path.join(output_dir, base_name + '.txt')

        with open(out_file, 'w') as f:
            for line in ann_list:
                f.write(line + '\n')
        count += 1

print(f"Processed {count} images with segmentation annotations")

Processed 1237 images with segmentation annotations


### Creation of Yaml file for the segmentation

In [ ]:
import yaml

# Define target directory for COCO dataset
output_dir = '/content/extracted_files/coco2017_subset/'

# Ensure the directory exists
os.makedirs(output_dir, exist_ok=True)

# COCO 2017 dataset configuration
data = {
    'path': output_dir,  # Base path to the dataset
    'train': '/content/extracted_files/coco2017_subset/train2017',  # Relative path to training images
    'val': '/content/extracted_files/coco2017_subset/val2017',  # Relative path to validation images
    'test': '/content/extracted_files/coco2017_subset/test2017',  # Relative path to test images (optional)

    'nc': 80,  # Number of classes in COCO dataset

    # Official COCO 2017 class names in order
    'names': [
        'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train',
        'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign',
        'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
        'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag',
        'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite',
        'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
        'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana',
        'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
        'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table',
        'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
        'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock',
        'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
    ]
}

# Path to save data.yaml
yaml_path = os.path.join(output_dir, 'seg.yaml')

# Write YAML file
with open(yaml_path, 'w') as file:
    yaml.dump(data, file, default_flow_style=False)

print(f"seg.yaml saved at: {yaml_path}")

seg.yaml saved at: /content/extracted_files/coco2017_subset/seg.yaml


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.0/950.0 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
from ultralytics import YOLO

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.version.cuda)

True
12.4


In [ ]:
model = YOLO("yolo11n-seg.pt")

In [ ]:
data_yaml='/content/extracted_files/coco2017_subset/seg.yaml'
epochs=3
img_size=640
batch_size=2

## Training the model  for segmentation

In [ ]:
print("Starting Yolo Training")
# Change device to 'cpu' if cuda is unavailable
res = model.train(data=data_yaml, epochs=epochs, imgsz=img_size, batch=batch_size, device=0)
print("Training completed")

Starting Yolo Training
Ultralytics 8.3.98 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=segment, mode=train, model=yolo11n-seg.pt, data=/content/extracted_files/coco2017_subset/seg.yaml, epochs=3, time=None, patience=100, batch=2, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=Fals

train: Scanning /content/extracted_files/coco2017_subset/train2017... 29315 images, 256 backgrounds, 0 corrupt: 100%|██████████| 29571/29571 [01:22<00:00, 360.56it/s]

train: WARNING ⚠️ /content/extracted_files/coco2017_subset/train2017/000000000315.jpg: 2 duplicate labels removed
train: WARNING ⚠️ /content/extracted_files/coco2017_subset/train2017/000000000540.jpg: 2 duplicate labels removed
train: WARNING ⚠️ /content/extracted_files/coco2017_subset/train2017/000000001941.jpg: 2 duplicate labels removed
train: WARNING ⚠️ /content/extracted_files/coco2017_subset/train2017/000000002184.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/extracted_files/coco2017_subset/train2017/000000002697.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/extracted_files/coco2017_subset/train2017/000000003378.jpg: 3 duplicate labels removed
train: WARNING ⚠️ /content/extracted_files/coco2017_subset/train2017/000000004592.jpg: 4 duplicate labels removed
train: WARNING ⚠️ /content/extracted_files/coco2017_subset/train2017/000000005376.jpg: 3 duplicate labels removed
train: WARNING ⚠️ /content/extracted_files/coco2017_subset/train2017/000000005688.jpg: 1

train: New cache created: /content/extracted_files/coco2017_subset/train2017.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/extracted_files/coco2017_subset/val2017... 1237 images, 13 backgrounds, 0 corrupt: 100%|██████████| 1250/1250 [00:03<00:00, 358.66it/s]

...

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):   1%|          | 2/313 [00:00<01:06,  4.69it/s]

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 313/313 [00:24<00:00, 12.70it/s]


                   all       1250      11125      0.368      0.297      0.273      0.155      0.316      0.131      0.121     0.0476
                person        670       3235      0.546      0.286      0.364      0.202       0.46     0.0971     0.0872     0.0271
               bicycle         34        102      0.377      0.225      0.206      0.107      0.326     0.0784     0.0668     0.0178
                   car        132        566      0.413      0.175      0.214     0.0925      0.406     0.0265     0.0241    0.00855
            motorcycle         36        124      0.433       0.29      0.295      0.162      0.403      0.129      0.106     0.0464
              airplane         23         46      0.262      0.609      0.435      0.232      0.172      0.152     0.0941     0.0402
                   bus         43         61      0.652      0.554      0.621      0.479      0.649       0.41      0.445      0.176
                 train         48         70       0.63      0.614   

## Validating the model

In [ ]:
results = model.val()  # Run validation

print(results.box.map)  # Mean Average Precision
print(results.box.map50)  # MAP at IoU 0.50
print(results.box.map75)  # MAP at IoU 0.75

Ultralytics 8.3.98 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n-seg summary (fused): 113 layers, 2,868,664 parameters, 0 gradients, 10.4 GFLOPs


val: Scanning /content/extracted_files/coco2017_subset/val2017.cache... 1237 images, 13 backgrounds, 0 corrupt: 100%|██████████| 1250/1250 [00:00<?, ?it/s]

val: WARNING ⚠️ /content/extracted_files/coco2017_subset/val2017/000000036494.jpg: 2 duplicate labels removed
val: WARNING ⚠️ /content/extracted_files/coco2017_subset/val2017/000000059635.jpg: 3 duplicate labels removed
val: WARNING ⚠️ /content/extracted_files/coco2017_subset/val2017/000000088040.jpg: 3 duplicate labels removed
val: WARNING ⚠️ /content/extracted_files/coco2017_subset/val2017/000000146155.jpg: 3 duplicate labels removed
val: WARNING ⚠️ /content/extracted_files/coco2017_subset/val2017/000000156278.jpg: 1 duplicate labels removed
val: WARNING ⚠️ /content/extracted_files/coco2017_subset/val2017/000000178982.jpg: 1 duplicate labels removed
val: WARNING ⚠️ /content/extracted_files/coco2017_subset/val2017/000000187144.jpg: 1 duplicate labels removed
val: WARNING ⚠️ /content/extracted_files/coco2017_subset/val2017/000000192670.jpg: 1 duplicate labels removed
val: WARNING ⚠️ /content/extracted_files/coco2017_subset/val2017/000000207844.jpg: 2 duplicate labels removed
val: WARNI


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):   0%|          | 0/625 [00:00<?, ?it/s]

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...


...

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):   0%|          | 2/625 [00:00<02:06,  4.94it/s]

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 625/625 [00:30<00:00, 20.20it/s]


                   all       1250      11125      0.365      0.296      0.273      0.155      0.315       0.13      0.121     0.0475
                person        670       3235      0.545      0.286      0.365      0.202      0.463     0.0974     0.0877     0.0272
               bicycle         34        102      0.367      0.216      0.202      0.107      0.326     0.0784      0.067     0.0179
                   car        132        566      0.412      0.175      0.213     0.0922      0.406     0.0265     0.0242    0.00855
            motorcycle         36        124      0.439       0.29      0.296      0.163      0.413      0.129      0.106     0.0464
              airplane         23         46      0.252      0.609      0.435      0.232      0.168      0.152     0.0941     0.0402
                   bus         43         61      0.639      0.541       0.62      0.479      0.649       0.41      0.445      0.173
                 train         48         70       0.63      0.614   

## Saving the model

In [ ]:
from google.colab import drive
import os
import shutil

# Mount Google Drive
drive.mount('/content/drive')

# Create a directory in your Drive for your models
save_dir = '/content/drive/MyDrive/yoloseg_models'
os.makedirs(save_dir, exist_ok=True)

# Copy your trained model to Google Drive
model_path = '/content/yolo11n-seg.pt'  # Your current model path
drive_model_path = f'{save_dir}/yolo11n-seg.pt'
shutil.copy(model_path, drive_model_path)

print(f"Model successfully saved to: {drive_model_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Model successfully saved to: /content/drive/MyDrive/yoloseg_models/yolo11n-seg.pt
